In [1]:
using Pkg; Pkg.activate(".")
using HypergraphModularity
using Printf
using StatsBase

 Activating environment at `~/code/hypergraph_modularities_code/Project.toml`
┌ Info: Precompiling HypergraphModularity [0c934d27-dd44-49d7-950f-bd4be7819e54]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `HypergraphModularity.ConstructAdj :: Tuple{SparseArrays.SparseMatrixCSC,Int64}` in module `HypergraphModularity`
└ @ Base.Docs docs/Docs.jl:227
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `HypergraphModularity.LamCCobj :: Tuple{SparseArrays.SparseMatrixCSC,Any,Any,Any}` in module `HypergraphModularity`
└ @ Base.Docs docs/Docs.jl:227
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `HypergraphModularity.renumber :: Tuple{Array{Int64,1},Array{Array{Int64,1},1}}` in module `HypergraphModularity`
└ @ Base.Docs docs/Docs.jl:227
  ** 

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [2]:
identity(p::Vector{Int64}) = p

function sum_of_ext_degs(p::Vector{Int64})
    soed = length(p) - 1
    return (sum(p), soed)
end

function all_or_nothing(p::Vector{Int64})
    is_aon = length(p) == 1
    return (sum(p), is_aon)
end

function rainbow(p::Vector{Int64})
    is_rainbow = length(p) == sum(p) && length(p) > 1
    return (sum(p), is_rainbow)
end

function all_or_nothing2(p::Vector{Int64})
    return length(p) == 1
end

all_or_nothing2 (generic function with 1 method)

In [3]:
function MLE_ll(Hyp, Z, agg)
    Ω̂ = estimateΩEmpirically(Hyp, Z; min_val=0, aggregator=agg)
    return Float64(sum(logLikelihood(Hyp, Z, Ω̂; α=0, bigInt=true)))    
end

function MLE_ll2(Hyp, Z, agg)
    Ω̂ = estimateΩEmpirically(Hyp, Z; min_val=0, aggregator=agg)
    return (Float64(sum(logLikelihood(Hyp, Z, Ω̂; α=0, bigInt=true))), Ω̂)    
end

function dyadic_MLE_ll(Hyp, Z, weighted::Bool, binary::Bool)
    ω_in, ω_out = 
        computeDyadicResolutionParameter(Hyp, Z; mode="ω", weighted=weighted, binary=binary)
    return Float64(dyadicLogLikelihood(Hyp, Z, ω_in, ω_out; weighted=weighted, binary=binary))
end

function dyadic_MLE_ll2(Hyp, Z, weighted::Bool, binary::Bool)
    ω_in, ω_out = 
        computeDyadicResolutionParameter(Hyp, Z; mode="ω", weighted=weighted, binary=binary)
    return (Float64(dyadicLogLikelihood(Hyp, Z, ω_in, ω_out; weighted=weighted, binary=binary)), ω_in, ω_out)
end

dyadic_MLE_ll2 (generic function with 1 method)

In [4]:
H, labels = read_hypergraph_data("walmart-trips", 10)
names = read_hypergraph_label_names("walmart-trips")

# throw out "Other" category
other_ind = findfirst(names .== "Other")
keep = Vector{Bool}(labels .!= other_ind)
(subH, node_map) = subhypergraph(H, keep)
sub_labels = zeros(Int64, maximum(subH.N))
for (i, j) in node_map
    sub_labels[j] = labels[i]
end

# Just to keep rest of the code the same
H = subH
labels = sub_labels
names = names[names .!= "Other"]
;

In [5]:
h_lls = Float64[]
g_lls = Float64[]
for z in 1:length(names)
    Z = ones(Int64, length(labels))
    Z[labels .== z] .= 2
    
    h_ll, Ω̂ = MLE_ll2(H, Z, all_or_nothing)
    g_ll, ω_in, ω_out = dyadic_MLE_ll2(H, Z, false, true)
        
    push!(h_lls, h_ll)
    push!(g_lls, g_ll)
    
    println(names[z], "...")
    println(@sprintf("graph: in=%.2E out=%.2E (%.2E) ll=%.2E", 
            ω_in, ω_out, ω_in / ω_out, g_ll))
    for k = 2:4
        ω_ink, ω_outk = Ω̂.ω((k, true), 0.0), Ω̂.ω((k, false), 0.0)
        println(@sprintf("hypergraph: k=%d in=%.2E out=%.2E (%.2E) ll=%.2E", 
                         k, ω_ink, ω_outk, ω_ink / ω_outk, h_ll))
    end
    println("------")
end

Clothing, Shoes & Accessories...
graph: in=1.11E+00 out=5.45E-01 (2.03E+00) ll=5.51E+04
hypergraph: k=2 in=3.16E-07 out=1.11E-07 (2.86E+00) ll=-1.59E+06
hypergraph: k=3 in=1.20E-12 out=4.68E-13 (2.56E+00) ll=-1.59E+06
hypergraph: k=4 in=4.44E-18 out=1.67E-18 (2.65E+00) ll=-1.59E+06
------
Electronics and Office...
graph: in=1.01E+00 out=7.22E-01 (1.40E+00) ll=2.08E+05
hypergraph: k=2 in=2.78E-07 out=2.37E-07 (1.17E+00) ll=-1.60E+06
hypergraph: k=3 in=1.00E-12 out=8.06E-13 (1.24E+00) ll=-1.60E+06
hypergraph: k=4 in=3.55E-18 out=2.24E-18 (1.58E+00) ll=-1.60E+06
------
Home, Furniture & Appliances...
graph: in=1.03E+00 out=7.05E-01 (1.46E+00) ll=1.93E+05
hypergraph: k=2 in=2.87E-07 out=1.70E-07 (1.69E+00) ll=-1.60E+06
hypergraph: k=3 in=1.04E-12 out=6.79E-13 (1.53E+00) ll=-1.60E+06
hypergraph: k=4 in=3.70E-18 out=2.10E-18 (1.76E+00) ll=-1.60E+06
------
Home Improvement & Patio...
graph: in=1.01E+00 out=6.24E-01 (1.63E+00) ll=1.17E+05
hypergraph: k=2 in=2.81E-07 out=1.41E-07 (1.99E+00) ll=

In [6]:
hsp = sortperm(h_lls, rev=true)
gsp = sortperm(g_lls, rev=true)
println(rpad("", 5), rpad("Hypergraph", 35 + 25), "\t", rpad("Graph", 30 + 15))
for (i, (j, k)) in enumerate(zip(hsp, gsp))
    h_ll = h_lls[j]
    g_ll = g_lls[k]
    println(rpad("$i: ", 5), 
            rpad("$(names[j])", 31),
            rpad(@sprintf("%.2E", h_ll), 15),
            rpad("$(names[k])", 31),
            rpad(@sprintf("%.2E", g_ll), 15))
end

     Hypergraph                                                  	Graph                                        
1:   Food, Household & Pets         -1.57E+06      Toys, Games, and Video Games   2.11E+05       
2:   Pharmacy, Health & Beauty      -1.59E+06      Electronics and Office         2.08E+05       
3:   Clothing, Shoes & Accessories  -1.59E+06      Home, Furniture & Appliances   1.93E+05       
4:   Baby                           -1.60E+06      Sports, Fitness & Outdoors     1.73E+05       
5:   Home, Furniture & Appliances   -1.60E+06      Auto, Tires & Industrial       1.72E+05       
6:   Auto, Tires & Industrial       -1.60E+06      Pharmacy, Health & Beauty      1.44E+05       
7:   Electronics and Office         -1.60E+06      Home Improvement & Patio       1.17E+05       
8:   Home Improvement & Patio       -1.60E+06      Food, Household & Pets         8.60E+04       
9:   Toys, Games, and Video Games   -1.60E+06      Baby                           8.17E+04       
10:  S

In [7]:
for sz = 2:5
    counts1 = zeros(Int64, maximum(labels))  # num edges of same type
    counts2 = zeros(Int64, maximum(labels))  # num total appearances
    for (e, _) in H.E[sz]
        e_labels = [labels[v] for v in e]
        if all(x -> x == e_labels[1], e_labels)
            counts1[e_labels[1]] += 1
        end
        counts2[e_labels] .+= 1
    end
    println("$sz ($(length(H.E[sz])) hedges)...")
    for (i, (cnt1, cnt2)) in enumerate(zip(counts1, counts2))
        rel_score = round(cnt1 / cnt2, digits=3)
        println(rpad("$(names[i])", 30), "\t", 
                rpad("$cnt1", 7), "\t", 
                rpad("$cnt2", 7), "\t",
                rpad("$rel_score", 7))
    end
    println("---------")
end

2 (12689 hedges)...
Clothing, Shoes & Accessories 	1188   	2188   	0.543  
Electronics and Office        	401    	1008   	0.398  
Home, Furniture & Appliances  	399    	1148   	0.348  
Home Improvement & Patio      	139    	374    	0.372  
Baby                          	174    	432    	0.403  
Toys, Games, and Video Games  	184    	527    	0.349  
Food, Household & Pets        	4412   	6787   	0.65   
Pharmacy, Health & Beauty     	1347   	2890   	0.466  
Sports, Fitness & Outdoors    	113    	386    	0.293  
Auto, Tires & Industrial      	359    	922    	0.389  
---------
3 (9702 hedges)...
Clothing, Shoes & Accessories 	539    	1898   	0.284  
Electronics and Office        	105    	768    	0.137  
Home, Furniture & Appliances  	138    	1102   	0.125  
Home Improvement & Patio      	70     	340    	0.206  
Baby                          	91     	404    	0.225  
Toys, Games, and Video Games  	62     	443    	0.14   
Food, Household & Pets        	3217   	6349   	0.507  
Pharmacy, Health

In [8]:
H, labels = read_hypergraph_data("TrivagoClickout", 10)
names = read_hypergraph_label_names("TrivagoClickout")
name_map = Dict(name => j for (j, name) in enumerate(names))
;

In [9]:
cm = countmap(labels)
scm = sort(collect(cm), by=x->x[2], rev=true)
tot = length(labels)
for (k, v) in scm[1:10]
    @show names[k], v
    tot -= v
end
@show tot

(names[k], v) = ("USA", 15519)
(names[k], v) = ("Brazil", 14921)
(names[k], v) = ("Japan", 10065)
(names[k], v) = ("India", 9963)
(names[k], v) = ("Italy", 9699)
(names[k], v) = ("Spain", 8654)
(names[k], v) = ("United Kingdom", 8430)
(names[k], v) = ("Germany", 8269)
(names[k], v) = ("Turkey", 6186)
(names[k], v) = ("Mexico", 5749)
tot = 75283


75283

In [10]:
#=
s_am = ["Brazil", "Colombia", "Argentina", "Peru", "Venezuela", "Chile", "Ecuador", "Bolivia",
        "Bolivia", "Paraguay", "Uruguay", "Guyana", "French Guiana"]
n_am = ["USA", "Canada", "Mexico"]
europe = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland",
          "France", "Germany", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", 
          "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden", "Turkey", 
          "United Kingdom", "Switzerland"]
asia = ["Japan", "Thailand", "Taiwan", "Indonesia", "Philippines", "Vietnam", "China"]

region_names = ["South America", "North America", "Europe", "Asia"]
=#

In [11]:
h_lls = Float64[]
g_lls = Float64[]
loc_names = []
#for region in [s_am, n_am, europe, asia]
#    inds = [name_map[c] for c in region]
for (k, _) in scm[1:10]
    Z = ones(Int64, length(labels))
    #Z[[l for l in labels if l in inds]] .= 2
    Z[labels .== k] .= 2
    push!(loc_names, names[k])
    
    h_ll = MLE_ll(H, Z, sum_of_ext_degs)
    g_ll = dyadic_MLE_ll(H, Z, false, false)
    
    push!(h_lls, h_ll)
    push!(g_lls, g_ll)    
end

In [12]:
hsp = sortperm(h_lls, rev=true)
gsp = sortperm(g_lls, rev=true)
println(rpad("",     5), rpad("Hypergraph",  35), "\t", rpad("Graph",       30))
for (i, (j, k)) in enumerate(zip(hsp, gsp))
    println(rpad("$i: ", 5), rpad("$(loc_names[j])", 35), "\t", rpad("$(loc_names[k])", 30))
end

     Hypergraph                         	Graph                         
1:   Brazil                             	Brazil                        
2:   Japan                              	USA                           
3:   USA                                	Japan                         
4:   United Kingdom                     	United Kingdom                
5:   Germany                            	Germany                       
6:   Turkey                             	Italy                         
7:   Spain                              	Spain                         
8:   Mexico                             	Turkey                        
9:   India                              	Mexico                        
10:  Italy                              	India                         
